In [1]:
import os
import json
os.environ["OPENAI_API_BASE"] = "https://api-proxy-openai.itwk.cc/v1"

In [2]:
import openai

def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo-0613", 
                                 temperature=0, max_tokens=600):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [3]:
def get_structed_data(prompt):
    if prompt == "":
        print("you did not enter travel prompt\n")
        prompt = "I want a one day trip from Shenzhen to Hong Kong"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f"{prompt}"\
                    }],
        functions=[
            {
                "name": "book_travel",
                "description": "Book travel",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination": {
                            "type": "string",
                            "description": "Your travel destination"
                        },
                        "departure": {
                            "type": "string",
                            "description": "From where are you traveling"
                        },
                        "days":{
                            "type": "string",
                            "description": "How many days does the travel has"
                        },
                        "travel_mode": {
                            "type": "string",
                            "description": "What mode of travel will it be, like direct or transit"
                        }
                    },
                    "required": ["destination", "departure"],
                },
            },
            {
                "name": "book_test",
                "description": "Book test",
                "parameters": {
                    "type": "object",
                    "properties": {
                    },
                    
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]
    print(message)
    if message.get("function_call"):
        function_args = json.loads(message["function_call"]["arguments"])
        print(function_args)
        #print(type(function_args))
    else:
        function_args = {}
        
    return function_args

In [4]:
def get_top15_attractions(mydest):
    user_message = f"""
    list the top20 attractions in {mydest}"""

    messages =  [  
    {'role':'system', 
     'content': "you are a travel Assistant to help find accurate travel information"},    
    {'role':'user', 
     'content': f"{user_message}"},  
    ] 

    response = get_completion_from_messages(messages)
    print(response)
    messages.append({"role":"assistant","content":response})
    return messages

In [5]:
def choose_attractions(messages):
    user_input = input("Please choose some attraction ids: ")
    if user_input == "":
        print("You entered an empty string. will use defconfig")
        user_input = "1,4,5,7,15"
    else:
        print(f"You entered: {user_input}")

    messages.append({"role":"user", "content": f"choose attraction {user_input} return attraction names in a json array"})
    print("check point 3", messages)
    response = get_completion_from_messages(messages)
    print(response)
    return response

In [6]:
def make_plan(attractions, days, messages):
    messages.append({"role":"assistant","content":attractions})
    messages.append({"role":"user", "content": f"based on these attractions above, plan {days} day trip for me, response with Chinese"})
    #step4 get the travel plan
    response = get_completion_from_messages(messages)
    #print(response)
    return response

In [7]:
# prompt: I want a one day trip from Shenzhen to Hong Kong
only_prompt = input("Please input travel prompt: example [I want a one day trip from Shenzhen to Hong Kong]")

#step1 extract infos by function call
function_args = get_structed_data(only_prompt)
mydest = function_args.get("destination")
mydays = function_args.get("days")
if mydays == "":
    mydays = "2"

#step2 get top15 attractions for a destination
messages = get_top15_attractions(mydest)

#step3 user choose attractions in list
response = choose_attractions(messages) 
#step4 get the travel plan
attractions_json = response
response = make_plan(attractions_json, mydays, messages)
print(response)

Please input travel prompt: example [I want a one day trip from Shenzhen to Hong Kong]
you did not enter travel prompt

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "book_travel",
    "arguments": "{\n  \"destination\": \"Hong Kong\",\n  \"departure\": \"Shenzhen\",\n  \"days\": \"1\"\n}"
  }
}
{'destination': 'Hong Kong', 'departure': 'Shenzhen', 'days': '1'}
1. Victoria Peak - Offers stunning views of the city skyline.
2. Hong Kong Disneyland - A magical theme park with various attractions and entertainment.
3. Tian Tan Buddha - A giant bronze statue located on Lantau Island.
4. Ocean Park - A marine-themed park with thrilling rides and animal exhibits.
5. Avenue of Stars - A waterfront promenade honoring Hong Kong's film industry.
6. Ngong Ping 360 - A cable car ride to the Big Buddha and Po Lin Monastery.
7. Temple Street Night Market - A bustling market with street food, souvenirs, and fortune tellers.
8. Hong Kong Museum of History - Showcasing the